# Estimation Tutorial for Stock Market Analytics

This notebook demonstrates how to use the estimation capabilities in the model factory for quantile regression and multi-target prediction.

## Overview

The main estimation components are:
1. **CatBoostMultiQuantileModel**: Scikit-learn compatible wrapper for multi-quantile regression
2. **Estimation Functions**: Helper utilities for feature engineering and data preparation
3. **Pipeline Integration**: Seamless integration with sklearn pipelines

Key features:
- **Multi-quantile prediction**: Simultaneous prediction of multiple quantiles
- **Automatic categorical feature detection**: Handles mixed data types
- **Monotonicity enforcement**: Ensures quantile ordering
- **Pipeline compatibility**: Works with sklearn preprocessing and cross-validation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import estimation classes
from stock_market_analytics.modeling.model_factory.estimation.estimators import (
    CatBoostMultiQuantileModel
)

# Import estimation helper functions
from stock_market_analytics.modeling.model_factory.estimation.estimation_functions import (
    detect_categorical_features,
    create_catboost_pool
)

# For data preprocessing and evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.compose import ColumnTransformer

# For creating synthetic financial data
import scipy.stats as stats

## 1. Creating Realistic Financial Dataset

We'll create a synthetic dataset that mimics real financial data with multiple asset types, market regimes, and mixed feature types:

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Create synthetic financial dataset
n_samples = 5000
n_symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN', 'JPM', 'BAC', 'XOM', 'KO', 'PFE']
sectors = ['Technology', 'Technology', 'Technology', 'Technology', 'Technology', 
          'Financial', 'Financial', 'Energy', 'Consumer', 'Healthcare']
market_caps = ['Large', 'Large', 'Large', 'Large', 'Large', 'Large', 'Large', 'Large', 'Large', 'Large']

# Generate time series data
start_date = pd.Timestamp('2020-01-01')
end_date = pd.Timestamp('2023-12-31')
date_range = pd.date_range(start_date, end_date, freq='D')

data_list = []

for i in range(n_samples):
    # Random date and symbol
    date = np.random.choice(date_range)
    symbol_idx = np.random.randint(len(n_symbols))
    symbol = n_symbols[symbol_idx]
    sector = sectors[symbol_idx]
    market_cap = market_caps[symbol_idx]
    
    # Create features with realistic financial relationships
    # Price-based features (normalized)
    returns_1d = np.random.normal(0.001, 0.02)  # Daily return
    returns_5d = returns_1d + np.random.normal(0, 0.01)  # 5-day return
    returns_20d = returns_5d + np.random.normal(0, 0.02)  # 20-day return
    
    # Volume features
    volume_ratio = np.random.lognormal(0, 0.5)  # Volume vs average
    volume_trend = np.random.normal(0, 0.3)  # Volume trend
    
    # Volatility features
    volatility_5d = np.random.gamma(2, 0.01)  # 5-day volatility
    volatility_20d = volatility_5d + np.random.gamma(1, 0.005)  # 20-day volatility
    
    # Market features
    market_return = np.random.normal(0.0005, 0.015)  # Market return
    sector_return = market_return + np.random.normal(0, 0.01)  # Sector return
    
    # Technical indicators
    rsi = np.random.uniform(20, 80)  # RSI indicator
    macd = np.random.normal(0, 0.001)  # MACD
    
    # Fundamental features (categorical)
    earnings_surprise = np.random.choice(['Beat', 'Meet', 'Miss'], p=[0.4, 0.3, 0.3])
    analyst_rating = np.random.choice(['Buy', 'Hold', 'Sell'], p=[0.5, 0.4, 0.1])
    
    # Create target: next day return with realistic dependencies
    # Base signal from momentum and mean reversion
    momentum_signal = 0.1 * returns_1d + 0.05 * returns_5d
    mean_reversion = -0.3 * returns_20d
    
    # Volatility effect
    vol_effect = np.random.normal(0, volatility_5d)
    
    # Sector and market effects
    market_effect = 0.5 * market_return
    sector_effect = 0.3 * (sector_return - market_return)
    
    # Categorical effects
    earnings_effect = {'Beat': 0.005, 'Meet': 0, 'Miss': -0.005}[earnings_surprise]
    rating_effect = {'Buy': 0.002, 'Hold': 0, 'Sell': -0.002}[analyst_rating]
    
    # Combine all effects with noise
    target = (
        momentum_signal +
        mean_reversion +
        vol_effect +
        market_effect +
        sector_effect +
        earnings_effect +
        rating_effect +
        np.random.normal(0, 0.01)  # Idiosyncratic noise
    )
    
    data_list.append({
        'date': date,
        'symbol': symbol,
        'sector': sector,
        'market_cap': market_cap,
        'returns_1d': returns_1d,
        'returns_5d': returns_5d,
        'returns_20d': returns_20d,
        'volume_ratio': volume_ratio,
        'volume_trend': volume_trend,
        'volatility_5d': volatility_5d,
        'volatility_20d': volatility_20d,
        'market_return': market_return,
        'sector_return': sector_return,
        'rsi': rsi,
        'macd': macd,
        'earnings_surprise': earnings_surprise,
        'analyst_rating': analyst_rating,
        'target': target
    })

# Create DataFrame
df = pd.DataFrame(data_list)

print(f"Dataset created with {len(df)} samples")
print(f"Features: {df.columns.tolist()[:-1]}")
print(f"Target: {df.columns[-1]}")
print(f"\nData types:")
print(df.dtypes)

# Basic statistics
print(f"\nTarget statistics:")
print(f"  Mean: {df['target'].mean():.6f}")
print(f"  Std: {df['target'].std():.6f}")
print(f"  Min: {df['target'].min():.6f}")
print(f"  Max: {df['target'].max():.6f}")

print(f"\nCategorical feature distributions:")
for col in ['sector', 'earnings_surprise', 'analyst_rating']:
    print(f"  {col}: {dict(df[col].value_counts())}")

## 2. Feature Engineering and Data Preparation

In [ ]:
# Separate features and target
feature_cols = [col for col in df.columns if col not in ['target', 'date']]
X = df[feature_cols].copy()
y = df['target'].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Detect categorical features automatically
categorical_features = detect_categorical_features(X)
numerical_features = [col for col in X.columns if col not in categorical_features]

print(f"\nAutomatic feature detection:")
print(f"  Categorical features ({len(categorical_features)}): {categorical_features}")
print(f"  Numerical features ({len(numerical_features)}): {numerical_features}")

# Add some engineered features
X['returns_momentum'] = X['returns_1d'] * X['returns_5d']  # Momentum interaction
X['vol_adjusted_return'] = X['returns_1d'] / (X['volatility_5d'] + 1e-8)  # Risk-adjusted return
X['relative_volume'] = np.log1p(X['volume_ratio'])  # Log-transformed volume
X['market_sector_spread'] = X['sector_return'] - X['market_return']  # Sector alpha

print(f"\nAfter feature engineering:")
print(f"  Total features: {X.shape[1]}")
print(f"  New features: returns_momentum, vol_adjusted_return, relative_volume, market_sector_spread")

# Update numerical features list
numerical_features.extend(['returns_momentum', 'vol_adjusted_return', 'relative_volume', 'market_sector_spread'])

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X['sector']
)

print(f"\nData split:")
print(f"  Training: {X_train.shape[0]} samples")
print(f"  Testing: {X_test.shape[0]} samples")

## 3. Basic Multi-Quantile Model Training

In [ ]:
# Define quantiles to predict
quantiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]

# Initialize CatBoost multi-quantile model
model = CatBoostMultiQuantileModel(
    quantiles=quantiles,
    random_state=42,
    verbose=False,  # Set to True to see training progress
    # CatBoost-specific parameters
    iterations=500,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    early_stopping_rounds=50
)

print(f"Model initialized with {len(quantiles)} quantiles: {quantiles}")
print(f"Categorical features will be auto-detected: {categorical_features}")

# Train the model
print("\nTraining multi-quantile model...")
model.fit(X_train, y_train)

print(f"Model training completed!")
print(f"  Best iteration: {model.best_iteration_}")
print(f"  Feature importances available: {hasattr(model, 'feature_importances_')}")

# Make predictions
print("\nGenerating predictions...")
y_pred_quantiles_train = model.predict(X_train)
y_pred_quantiles_test = model.predict(X_test)

print(f"Training predictions shape: {y_pred_quantiles_train.shape}")
print(f"Test predictions shape: {y_pred_quantiles_test.shape}")

# Check prediction monotonicity
def check_monotonicity(predictions):
    """Check if quantile predictions are monotonic."""
    violations = 0
    for i in range(predictions.shape[0]):
        if not np.all(np.diff(predictions[i, :]) >= 0):
            violations += 1
    return violations, violations / predictions.shape[0]

train_violations, train_violation_rate = check_monotonicity(y_pred_quantiles_train)
test_violations, test_violation_rate = check_monotonicity(y_pred_quantiles_test)

print(f"\nMonotonicity check:")
print(f"  Training violations: {train_violations} ({train_violation_rate:.3%})")
print(f"  Test violations: {test_violations} ({test_violation_rate:.3%})")

## 4. Model Performance Analysis

In [ ]:
# Calculate coverage for each quantile
def calculate_coverage(y_true, y_pred_quantiles, quantiles):
    """Calculate empirical coverage for each quantile."""
    coverage = []
    for i, q in enumerate(quantiles):
        empirical_coverage = np.mean(y_true <= y_pred_quantiles[:, i])
        coverage.append(empirical_coverage)
    return np.array(coverage)

# Calculate pinball loss for each quantile
def pinball_loss(y_true, y_pred, quantile):
    """Calculate pinball loss for a specific quantile."""
    error = y_true - y_pred
    return np.mean(np.maximum(quantile * error, (quantile - 1) * error))

# Evaluate model performance
coverage_train = calculate_coverage(y_train, y_pred_quantiles_train, quantiles)
coverage_test = calculate_coverage(y_test, y_pred_quantiles_test, quantiles)

# Calculate pinball losses
pinball_losses_train = []
pinball_losses_test = []

for i, q in enumerate(quantiles):
    train_loss = pinball_loss(y_train, y_pred_quantiles_train[:, i], q)
    test_loss = pinball_loss(y_test, y_pred_quantiles_test[:, i], q)
    pinball_losses_train.append(train_loss)
    pinball_losses_test.append(test_loss)

pinball_losses_train = np.array(pinball_losses_train)
pinball_losses_test = np.array(pinball_losses_test)

# Performance summary
print("Quantile Regression Performance Analysis:")
print("=" * 70)
print(f"{'Quantile':<10} {'Target':<8} {'Train Cov':<10} {'Test Cov':<10} {'Train PB':<10} {'Test PB':<10}")
print("=" * 70)

for i, q in enumerate(quantiles):
    print(f"{q:<10.2f} {q:<8.2f} {coverage_train[i]:<10.3f} {coverage_test[i]:<10.3f} "
          f"{pinball_losses_train[i]:<10.6f} {pinball_losses_test[i]:<10.6f}")

print("=" * 70)
print(f"{'Mean':<10} {'':<8} {np.mean(coverage_train):<10.3f} {np.mean(coverage_test):<10.3f} "
      f"{np.mean(pinball_losses_train):<10.6f} {np.mean(pinball_losses_test):<10.6f}")

# Coverage errors
coverage_errors_train = np.abs(coverage_train - quantiles)
coverage_errors_test = np.abs(coverage_test - quantiles)

print(f"\nCoverage Error Analysis:")
print(f"  Training - Mean: {np.mean(coverage_errors_train):.4f}, Max: {np.max(coverage_errors_train):.4f}")
print(f"  Test - Mean: {np.mean(coverage_errors_test):.4f}, Max: {np.max(coverage_errors_test):.4f}")

# Model scoring (using median quantile)
median_idx = len(quantiles) // 2
r2_train = model.score(X_train, y_train)
r2_test = model.score(X_test, y_test)

print(f"\nModel R² Score (median quantile):")
print(f"  Training: {r2_train:.4f}")
print(f"  Test: {r2_test:.4f}")

## 5. Visualization of Quantile Predictions

In [ ]:
# Create comprehensive visualizations
fig = plt.figure(figsize=(16, 12))

# Plot 1: Coverage analysis
ax1 = plt.subplot(2, 3, 1)
x_pos = np.arange(len(quantiles))
width = 0.35

bars1 = ax1.bar(x_pos - width/2, coverage_train, width, label='Training', alpha=0.7)
bars2 = ax1.bar(x_pos + width/2, coverage_test, width, label='Test', alpha=0.7)
ax1.plot(x_pos, quantiles, 'ro-', label='Target', linewidth=2, markersize=6)

ax1.set_xlabel('Quantiles')
ax1.set_ylabel('Empirical Coverage')
ax1.set_title('Quantile Coverage Analysis')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([f'{q:.2f}' for q in quantiles], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Pinball loss comparison
ax2 = plt.subplot(2, 3, 2)
ax2.bar(x_pos - width/2, pinball_losses_train, width, label='Training', alpha=0.7)
ax2.bar(x_pos + width/2, pinball_losses_test, width, label='Test', alpha=0.7)

ax2.set_xlabel('Quantiles')
ax2.set_ylabel('Pinball Loss')
ax2.set_title('Pinball Loss by Quantile')
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f'{q:.2f}' for q in quantiles], rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Feature importance
ax3 = plt.subplot(2, 3, 3)
feature_importance = model.feature_importances_
feature_names = X_train.columns

# Sort by importance
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=True)

# Show top 10 features
top_features = importance_df.tail(10)
ax3.barh(range(len(top_features)), top_features['importance'], alpha=0.7)
ax3.set_yticks(range(len(top_features)))
ax3.set_yticklabels(top_features['feature'])
ax3.set_xlabel('Feature Importance')
ax3.set_title('Top 10 Feature Importances')
ax3.grid(True, alpha=0.3)

# Plot 4: Prediction intervals visualization (sample)
ax4 = plt.subplot(2, 3, 4)
n_viz = 100
viz_idx = np.random.choice(len(y_test), n_viz, replace=False)
viz_idx = np.sort(viz_idx)

y_test_viz = y_test.iloc[viz_idx]
y_pred_viz = y_pred_quantiles_test[viz_idx, :]

# Sort by median prediction for better visualization
median_pred = y_pred_viz[:, median_idx]
sort_idx = np.argsort(median_pred)

x_viz = np.arange(len(sort_idx))

# Plot prediction intervals (10%-90%, 25%-75%)
q10_idx, q25_idx, q75_idx, q90_idx = 1, 2, 4, 5  # Indices for quantiles

ax4.fill_between(x_viz, y_pred_viz[sort_idx, q10_idx], y_pred_viz[sort_idx, q90_idx], 
                alpha=0.2, color='lightblue', label='10%-90% Interval')
ax4.fill_between(x_viz, y_pred_viz[sort_idx, q25_idx], y_pred_viz[sort_idx, q75_idx], 
                alpha=0.3, color='lightgreen', label='25%-75% Interval')
ax4.plot(x_viz, y_pred_viz[sort_idx, median_idx], 'b-', label='Median Prediction', linewidth=2)
ax4.scatter(x_viz, y_test_viz.iloc[sort_idx], color='red', s=20, alpha=0.7, label='True Values')

ax4.set_xlabel('Sample Index (sorted)')
ax4.set_ylabel('Target Value')
ax4.set_title('Prediction Intervals Visualization')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Residuals analysis
ax5 = plt.subplot(2, 3, 5)
median_pred_test = y_pred_quantiles_test[:, median_idx]
residuals = y_test - median_pred_test

ax5.scatter(median_pred_test, residuals, alpha=0.6, s=10)
ax5.axhline(y=0, color='red', linestyle='--', alpha=0.7)
ax5.set_xlabel('Predicted Values (Median)')
ax5.set_ylabel('Residuals')
ax5.set_title('Residuals vs Predicted')
ax5.grid(True, alpha=0.3)

# Plot 6: QQ plot for residuals
ax6 = plt.subplot(2, 3, 6)
stats.probplot(residuals, dist="norm", plot=ax6)
ax6.set_title('Q-Q Plot of Residuals')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Pipeline Integration and Cross-Validation

In [ ]:
# Create a preprocessing pipeline that works with mixed data types
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer

# Note: CatBoost handles categorical features automatically, but let's show integration
# For demonstration, we'll create a simple preprocessor
preprocessor = make_column_transformer(
    (StandardScaler(), numerical_features),
    remainder='passthrough'  # Keep categorical features as-is for CatBoost
)

# Create pipeline with preprocessing and model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostMultiQuantileModel(
        quantiles=[0.1, 0.25, 0.5, 0.75, 0.9],
        random_state=42,
        verbose=False,
        iterations=300,
        learning_rate=0.1
    ))
])

print("Pipeline created with preprocessing and CatBoost model")
print(f"Numerical features to be scaled: {len(numerical_features)}")
print(f"Categorical features (passthrough): {len(categorical_features)}")

# Fit the pipeline
print("\nTraining pipeline...")
pipeline.fit(X_train, y_train)

# Make predictions
y_pred_pipeline = pipeline.predict(X_test)
print(f"Pipeline predictions shape: {y_pred_pipeline.shape}")

# Score the pipeline (uses median quantile by default)
pipeline_score = pipeline.score(X_test, y_test)
print(f"Pipeline R² score: {pipeline_score:.4f}")

# Cross-validation with the pipeline
print("\nPerforming cross-validation...")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='r2')
print(f"CV R² scores: {cv_scores}")
print(f"CV mean ± std: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Extract feature importance from the pipeline
model_from_pipeline = pipeline.named_steps['model']
feature_importance_pipeline = model_from_pipeline.feature_importances_
print(f"\nFeature importances extracted from pipeline: {len(feature_importance_pipeline)} features")

## 7. Hyperparameter Tuning

In [ ]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    'model__iterations': [200, 300, 500],
    'model__learning_rate': [0.05, 0.1, 0.15],
    'model__depth': [4, 6, 8],
    'model__l2_leaf_reg': [1, 3, 5]
}

# Create a simpler pipeline for faster tuning
tuning_pipeline = Pipeline([
    ('model', CatBoostMultiQuantileModel(
        quantiles=[0.1, 0.5, 0.9],  # Fewer quantiles for faster tuning
        random_state=42,
        verbose=False,
        early_stopping_rounds=30
    ))
])

print("Starting hyperparameter tuning...")
print(f"Parameter grid: {len(param_grid)} parameters")
print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")

# Perform grid search with reduced scope for demonstration
# In practice, you might use RandomizedSearchCV for larger grids
grid_search = GridSearchCV(
    tuning_pipeline,
    param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

# Fit on a subset for demonstration (full dataset would take longer)
sample_size = min(2000, len(X_train))
sample_idx = np.random.choice(len(X_train), sample_size, replace=False)
X_train_sample = X_train.iloc[sample_idx]
y_train_sample = y_train.iloc[sample_idx]

print(f"\nTuning on sample of {sample_size} training examples...")
grid_search.fit(X_train_sample, y_train_sample)

print(f"\nGrid search completed!")
print(f"Best score: {grid_search.best_score_:.4f}")
print(f"Best parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Evaluate best model on full test set
best_model = grid_search.best_estimator_
best_score_test = best_model.score(X_test, y_test)
print(f"\nBest model test score: {best_score_test:.4f}")

# Compare with default parameters
print(f"\nComparison:")
print(f"  Default model test score: {pipeline_score:.4f}")
print(f"  Tuned model test score: {best_score_test:.4f}")
print(f"  Improvement: {((best_score_test - pipeline_score) / abs(pipeline_score) * 100):.2f}%")

## 8. Advanced Analysis: Sector-Specific Performance

In [ ]:
# Analyze model performance by sector
X_test_with_sector = X_test.copy()
y_pred_full = model.predict(X_test)  # Use the original full model

# Get median predictions for sector analysis
median_pred = y_pred_full[:, median_idx]

# Calculate metrics by sector
sectors = X_test_with_sector['sector'].unique()
sector_metrics = {}

for sector in sectors:
    sector_mask = X_test_with_sector['sector'] == sector
    if sector_mask.sum() > 10:  # Only analyze sectors with sufficient data
        y_true_sector = y_test[sector_mask]
        y_pred_sector = median_pred[sector_mask]
        y_quantiles_sector = y_pred_full[sector_mask, :]
        
        # Calculate metrics
        mse = mean_squared_error(y_true_sector, y_pred_sector)
        mae = mean_absolute_error(y_true_sector, y_pred_sector)
        
        # Calculate coverage for 25%-75% interval
        q25_idx, q75_idx = 2, 4
        coverage_50 = np.mean(
            (y_true_sector >= y_quantiles_sector[:, q25_idx]) & 
            (y_true_sector <= y_quantiles_sector[:, q75_idx])
        )
        
        sector_metrics[sector] = {
            'n_samples': sector_mask.sum(),
            'mse': mse,
            'mae': mae,
            'rmse': np.sqrt(mse),
            'coverage_50': coverage_50,
            'mean_prediction': y_pred_sector.mean(),
            'mean_actual': y_true_sector.mean()
        }

# Display sector analysis
print("Sector-Specific Performance Analysis:")
print("=" * 80)
print(f"{'Sector':<12} {'N':<6} {'RMSE':<8} {'MAE':<8} {'Coverage':<8} {'Pred Mean':<10} {'Actual Mean':<10}")
print("=" * 80)

for sector, metrics in sector_metrics.items():
    print(f"{sector:<12} {metrics['n_samples']:<6} {metrics['rmse']:<8.5f} {metrics['mae']:<8.5f} "
          f"{metrics['coverage_50']:<8.2%} {metrics['mean_prediction']:<10.5f} {metrics['mean_actual']:<10.5f}")

print("=" * 80)

# Visualize sector performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: RMSE by sector
sectors_list = list(sector_metrics.keys())
rmse_values = [sector_metrics[s]['rmse'] for s in sectors_list]

axes[0, 0].bar(sectors_list, rmse_values, alpha=0.7)
axes[0, 0].set_title('RMSE by Sector')
axes[0, 0].set_ylabel('RMSE')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Coverage by sector
coverage_values = [sector_metrics[s]['coverage_50'] for s in sectors_list]
axes[0, 1].bar(sectors_list, coverage_values, alpha=0.7, color='green')
axes[0, 1].axhline(y=0.5, color='red', linestyle='--', label='Target (50%)')
axes[0, 1].set_title('50% Interval Coverage by Sector')
axes[0, 1].set_ylabel('Coverage')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Prediction vs Actual means by sector
pred_means = [sector_metrics[s]['mean_prediction'] for s in sectors_list]
actual_means = [sector_metrics[s]['mean_actual'] for s in sectors_list]

x_pos = np.arange(len(sectors_list))
width = 0.35

axes[1, 0].bar(x_pos - width/2, pred_means, width, label='Predicted', alpha=0.7)
axes[1, 0].bar(x_pos + width/2, actual_means, width, label='Actual', alpha=0.7)
axes[1, 0].set_title('Mean Values by Sector')
axes[1, 0].set_ylabel('Mean Return')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(sectors_list, rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Sample sizes by sector
sample_sizes = [sector_metrics[s]['n_samples'] for s in sectors_list]
axes[1, 1].bar(sectors_list, sample_sizes, alpha=0.7, color='orange')
axes[1, 1].set_title('Sample Size by Sector')
axes[1, 1].set_ylabel('Number of Samples')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Model Interpretation and Feature Analysis

In [ ]:
# Feature importance analysis
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 Most Important Features:")
print("=" * 40)
for i, row in feature_importance_df.head(15).iterrows():
    print(f"{row['feature']:<25} {row['importance']:<10.4f}")

# Analyze feature importance by category
feature_categories = {
    'Returns': ['returns_1d', 'returns_5d', 'returns_20d', 'returns_momentum'],
    'Volume': ['volume_ratio', 'volume_trend', 'relative_volume'],
    'Volatility': ['volatility_5d', 'volatility_20d', 'vol_adjusted_return'],
    'Market': ['market_return', 'sector_return', 'market_sector_spread'],
    'Technical': ['rsi', 'macd'],
    'Categorical': ['symbol', 'sector', 'market_cap', 'earnings_surprise', 'analyst_rating']
}

category_importance = {}
for category, features in feature_categories.items():
    category_features = [f for f in features if f in feature_importance_df['feature'].values]
    if category_features:
        category_imp = feature_importance_df[feature_importance_df['feature'].isin(category_features)]['importance'].sum()
        category_importance[category] = category_imp

print(f"\nFeature Importance by Category:")
print("=" * 35)
for category, importance in sorted(category_importance.items(), key=lambda x: x[1], reverse=True):
    print(f"{category:<15} {importance:<10.4f} ({importance/sum(category_importance.values())*100:.1f}%)")

# Create feature importance visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Top features
top_features = feature_importance_df.head(12)
axes[0].barh(range(len(top_features)), top_features['importance'], alpha=0.7)
axes[0].set_yticks(range(len(top_features)))
axes[0].set_yticklabels(top_features['feature'])
axes[0].set_xlabel('Feature Importance')
axes[0].set_title('Top 12 Feature Importances')
axes[0].grid(True, alpha=0.3)

# Category importance
categories = list(category_importance.keys())
importances = list(category_importance.values())
colors = plt.cm.Set3(np.linspace(0, 1, len(categories)))

wedges, texts, autotexts = axes[1].pie(importances, labels=categories, autopct='%1.1f%%', 
                                      colors=colors, startangle=90)
axes[1].set_title('Feature Importance by Category')

plt.tight_layout()
plt.show()

# Analyze predictions by categorical features
print(f"\nPrediction Analysis by Categorical Features:")
print("=" * 50)

for cat_feature in ['sector', 'earnings_surprise', 'analyst_rating']:
    if cat_feature in X_test.columns:
        print(f"\n{cat_feature.upper()}:")
        for category in X_test[cat_feature].unique():
            mask = X_test[cat_feature] == category
            if mask.sum() > 5:  # Only show categories with sufficient samples
                actual_mean = y_test[mask].mean()
                pred_mean = median_pred[mask].mean()
                n_samples = mask.sum()
                print(f"  {category:<15}: Actual={actual_mean:>8.5f}, Pred={pred_mean:>8.5f}, N={n_samples:>3}")

## 10. Summary and Best Practices

### Key Takeaways:

1. **CatBoostMultiQuantileModel** provides a robust sklearn-compatible interface for multi-quantile regression
2. **Automatic categorical feature detection** simplifies data preprocessing
3. **Pipeline integration** enables seamless use with sklearn's preprocessing and cross-validation tools
4. **Multi-quantile prediction** captures uncertainty and provides rich distributional information

### Best Practices:

1. **Feature Engineering**: Create domain-specific features (momentum, volatility-adjusted returns, etc.)
2. **Categorical Features**: Let CatBoost handle categorical features automatically for optimal performance
3. **Quantile Selection**: Choose quantiles based on your specific use case (risk management, portfolio optimization)
4. **Cross-Validation**: Use proper CV strategies, especially for time series data
5. **Hyperparameter Tuning**: Balance model complexity with overfitting risk
6. **Domain Analysis**: Analyze performance by relevant segments (sectors, market conditions, etc.)

### Model Deployment Considerations:

1. **Monitoring**: Track prediction quality and coverage over time
2. **Retraining**: Establish retraining schedules based on data drift detection
3. **Feature Stability**: Monitor feature importance changes and distribution shifts
4. **Performance Metrics**: Use appropriate metrics for quantile regression (pinball loss, coverage)

In [ ]:
# Final model summary
print("FINAL MODEL ESTIMATION SUMMARY")
print("=" * 50)

print(f"Dataset: {len(df)} samples, {X.shape[1]} features")
print(f"  Training: {len(X_train)} samples")
print(f"  Testing: {len(X_test)} samples")

print(f"\nModel Configuration:")
print(f"  Quantiles: {model.quantiles}")
print(f"  Iterations: {model.best_iteration_}")
print(f"  Categorical features: {len(categorical_features)}")
print(f"  Numerical features: {len(numerical_features)}")

print(f"\nPerformance (R² Score):")
print(f"  Training: {model.score(X_train, y_train):.4f}")
print(f"  Test: {model.score(X_test, y_test):.4f}")

print(f"\nQuantile Coverage Analysis:")
print(f"  Mean coverage error (train): {np.mean(coverage_errors_train):.4f}")
print(f"  Mean coverage error (test): {np.mean(coverage_errors_test):.4f}")
print(f"  Max coverage error (test): {np.max(coverage_errors_test):.4f}")

print(f"\nTop 3 Most Important Features:")
for i, row in feature_importance_df.head(3).iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

print(f"\nMonotonicity Violations:")
print(f"  Training: {train_violation_rate:.3%}")
print(f"  Test: {test_violation_rate:.3%}")

print("\n✅ Estimation tutorial completed successfully!")